*Imports*

In [1]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder

from sklearn import metrics

Configurations

In [2]:
# pandas will display all data in a single column
#pd.set_option('display.max.columns', None);
# set the precision of floating point numbers
#pd.set_option('display.precision', 2);

Load the data

In [3]:
df = pd.read_csv('./train.csv');

_Common functions_

In [4]:
# execute and log
def eal(headerMsg, result):
    print('-----------'+headerMsg+'---------');
    print(result);
    print(' ');
  
"""
def oheCols(data, columns):
    
    for col in columns:
        encoder = LabelBinarizer();
        encoder.fit(data[col].astype(str));
        transformed = encoder.transform(data[col].astype(str));
        oheDf = pd.DataFrame(transformed);
        data = pd.concat([data, oheDf], axis=1).drop([col], axis=1);

        
    return data;
"""

'\ndef oheCols(data, columns):\n    \n    for col in columns:\n        encoder = LabelBinarizer();\n        encoder.fit(data[col].astype(str));\n        transformed = encoder.transform(data[col].astype(str));\n        oheDf = pd.DataFrame(transformed);\n        data = pd.concat([data, oheDf], axis=1).drop([col], axis=1);\n\n        \n    return data;\n'

Inspecting data

In [5]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
eal('number of samples', len(df));
eal('shape', df.shape);

-----------number of samples---------
891
 
-----------shape---------
(891, 12)
 


In [7]:
# checking data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Preparing data

In [8]:
y = df['Survived']
# removing some features (feature selection)
X = df.drop(['Survived', 'Name', 'Cabin', 'Ticket', 'Embarked', 'PassengerId'], 1)
X = X.fillna(X.mean())

C:\Users\Pichau\AppData\Local\Temp/ipykernel_14088/2942453561.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = df.drop(['Survived', 'Name', 'Cabin', 'Ticket', 'Embarked', 'PassengerId'], 1)
C:\Users\Pichau\AppData\Local\Temp/ipykernel_14088/2942453561.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X = X.fillna(X.mean())


In [9]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,male,22.0,1,0,7.2500
1,1,female,38.0,1,0,71.2833
2,3,female,26.0,0,0,7.9250
3,1,female,35.0,1,0,53.1000
4,3,male,35.0,0,0,8.0500


In [10]:
# One hot encoding 

colsToOneHotEncode = ['Sex'];

myEncoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
myEncoder.fit(X[colsToOneHotEncode])

oheX = pd.concat(
    [X.drop(colsToOneHotEncode, 1), 
     pd.DataFrame(myEncoder.transform(X[colsToOneHotEncode]))], axis=1).reindex()




C:\Users\Pichau\AppData\Local\Temp/ipykernel_14088/1321783184.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  [X.drop(colsToOneHotEncode, 1),


In [11]:
print(oheX.head())

   Pclass   Age  SibSp  Parch     Fare    0    1
0       3  22.0      1      0   7.2500  0.0  1.0
1       1  38.0      1      0  71.2833  1.0  0.0
2       3  26.0      0      0   7.9250  1.0  0.0
3       1  35.0      1      0  53.1000  1.0  0.0
4       3  35.0      0      0   8.0500  0.0  1.0


Splitting train and validation

In [12]:
X_train, X_test, y_train, y_test = train_test_split(oheX, y, test_size=0.3, random_state=1)

In [13]:
print(X_train.shape)
print(X_test.shape)
print('--')
print(y_train.shape)
print(y_test.shape)

(623, 7)
(268, 7)
--
(623,)
(268,)


# Baseline - Decision Tree Classifier

In [14]:
clf = DecisionTreeClassifier()

# Training decision tree classifier
clf = clf.fit(X_train,y_train)

# predict
y_pred = clf.predict(X_test)

In [15]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.746268656716418
